# Putting it all together

So far we have done the following on the prior Notebooks:

- **Notebook 01**: We loaded the Azure Search Engine with enriched PDFs in index: "cogsrch-index-files"
- **Notebook 02**: We loaded more information to the Search Engine this time using a CSV file with 52k rows/articles in index: "cogsrch-index-csv"
- **Notebook 03**: We added AzureOpenAI GPT models to enhance the the production of the answer by using Utility Chains of LLMs
- **Notebook 04**: We loaded a vector-based index with large/complex PDFs information , "cogsrch-index-books-vector"
- **Notebook 05**: We added memory to our system in order to power a conversational Chat Bot
- **Notebook 06**: We introduced Agents and Tools in order to be able to solve a more complex task: ask questions to Tabular datasets
- **Notebook 07**: We used a SQL Agent in order to talk to a SQL Database directly
- **Notebook 08**: We used another ReAct Agent in order to talk to the Bing Search API and create a Bing Chat Clone and implemented callbacks for real-time streaming and tool information


We are missing one more thing: **How do we glue all these features together into a very smart GPT Smart Search Engine Chat Bot?**

We want a virtual assistant for our company that can get the question, think what tool to use, then get the answer. The goal is that, regardless of the source of the information (Search Engine, Bing Search, SQL Database, CSV File, JSON File, etc), the Assistant can answer the question correctly using the right tool.

In this Notebook we are going to create that "brain" Agent, that will understand the question and use the right tool to get the answer from the right source.

Let's go..

In [ ]:
import os
import random
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import ConversationalChatAgent, AgentExecutor, Tool
from langchain.memory import CosmosDBChatMessageHistory
from langchain.callbacks.manager import CallbackManager

#custom libraries that we will use later in the app
from common.utils import DocSearchTool, CSVTabularTool, SQLDbTool, ChatGPTTool, BingSearchTool, run_agent
from common.callbacks import StdOutCallbackHandler
from common.prompts import CUSTOM_CHATBOT_PREFIX, CUSTOM_CHATBOT_SUFFIX 

from dotenv import load_dotenv
load_dotenv("my_credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))

MODEL_DEPLOYMENT_NAME = "gpt-4-32k" # Reminder: gpt-35-turbo models will create parsing errors and won't follow instructions correctly 

In [ ]:
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

### Get the Tools - Doc Search, CSV Agent, SQL Agent and  Web Search

In the file `common/utils.py` we created Agent Tools Classes for each of the Functionalities that we developed in prior Notebooks. This means that we are not using `qa_with_sources` chain anymore as we did until notebook 5. Agents that Reason, Act and Reflect is the best way to create bots that comunicate with sources.

In [ ]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0.5, max_tokens=1000)

# Uncomment the below line if you want to see the responses being streamed/typed
# llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0.5, max_tokens=500, streaming=True, callback_manager=cb_manager)

In [ ]:
# DocSearchTool is our Custom Tool Class (Agent) created for Azure Cognitive Search + OpenAI searches
text_indexes = ["cogsrch-index-files", "cogsrch-index-csv"]
doc_search = DocSearchTool(llm=llm, indexes=text_indexes,
                           k=10, similarity_k=4, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           callback_manager=cb_manager, return_direct=True)

In [ ]:
vector_only_indexes = ["cogsrch-index-books-vector"]
book_search = DocSearchTool(llm=llm, vector_only_indexes = vector_only_indexes,
                           k=10, similarity_k=10, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           callback_manager=cb_manager, return_direct=True,
                           # This is how you can edit the default values of name and description
                           name="@booksearch",
                           description="useful when the questions includes the term: @booksearch.\n")

In [ ]:
# BingSearchTool is a langchain Tool class to use the Bing Search API (https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)
www_search = BingSearchTool(llm=llm, k=5, callback_manager=cb_manager, return_direct=True)

In [ ]:
## CSVTabularTool is a custom Tool class crated to Q&A over CSV files
file_url = "./data/covid/all-states-history.csv"
csv_search = CSVTabularTool(path=file_url, llm=llm, callback_manager=cb_manager, return_direct=True)

In [ ]:
## SQLDbTool is a custom Tool class created to Q&A over a MS SQL Database
sql_search = SQLDbTool(llm=llm, k=30, callback_manager=cb_manager, return_direct=True)

In [ ]:
## ChatGPTTool is a custom Tool class created to talk to ChatGPT knowledge
chatgpt_search = ChatGPTTool(llm=llm, callback_manager=cb_manager, return_direct=True)

### Variables/knobs to use for customization

As you have seen so far, there are many knobs that you can dial up or down in order to change the behavior of your GPT Smart Search engine application, these are the variables you can tune:

- <u>llm</u>:
  - **deployment_name**: this is the deployment name of your Azure OpenAI model. This of course dictates the level of reasoning and the amount of tokens available for the conversation. For a production system you will need gpt-4-32k. This is the model that will give you enough reasoning power to work with agents, and enough tokens to work with detailed answers and conversation memory.
  - **temperature**: How creative you want your responses to be
  - **max_tokens**: How long you want your responses to be. It is recommended a minimum of 500
- <u>Tools</u>: To each tool you can add the following parameters to modify the defaults (set in utils.py), these are very important since they are part of the system prompt and determines what tool to use and when.
  - **name**: the name of the tool
  - **description**: when the brain agent should use this tool
- <u>DocSearchTool</u>: 
  - **k**: The top k results per index from the text search action
  - **similarity_k**: top k results combined from the vector search action
  - **reranker_th**: threshold of the semantic search reranker. Picks results that are above the threshold. Max possible score=4
- <u>BingSearchTool</u>:
  - **k**: The top k results from the bing search action
- <u>SQLDBTool</u>:
  - **k**: The top k results from the SQL search action. Adds TOP clause to the query
  
in `utils.py` you can also tune:
- <u>model_tokens_limit</u>: In this function you can edit what is the maximum allows of tokens reserve for the content. Remember that the remaining will be for the system prompt plus the answer

### Test the Tools

In [ ]:
# Test the Documents Search Tool with a question we know it doesn't have the knowledge for
printmd(doc_search.run("what is the weather today in Dallas?"))

In [ ]:
# Test the Document Search Tool with a question that we know it has the answer for
printmd(doc_search.run("How Covid affects obese people? and elderly?"))

In [ ]:
printmd(book_search.run("What's the acronim of the main point of the book Made to Stick"))

In [ ]:
# Test the Bing Search Tool
printmd(www_search.run("Who are the family member names of the current president of India?"))

In [ ]:
# Test the CSV Tool
printmd(csv_search.run("how many rows does the file have?"))

In [ ]:
# Test the SQL Search Tool
printmd(sql_search.run("How many people in total died california in each state of the west coast in July 2020?"))

In [ ]:
# Test the ChatGPTWrapper Search Tool
printmd(chatgpt_search.run("what is the function in python that allows me to get a random number?"))

### Define what tools are we going to give to our brain agent

Go to `common/utils.py` to check the tools definition and the instructions on what tool to use when

In [ ]:
tools = [www_search, sql_search, doc_search, book_search, chatgpt_search]

**Note**: Notice that since both the CSV file and the SQL Database have the same exact data, we are only going to use the SQLDBTool since it is faster and more reliable

### Initialize the brain agent

In [ ]:
cosmos = CosmosDBChatMessageHistory(
    cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
    cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
    cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
    connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
    session_id="Agent-Test-Session" + str(random.randint(1, 1000)),
    user_id="Agent-Test-User" + str(random.randint(1, 1000))
    )
# prepare the cosmosdb instance
cosmos.prepare_cosmos()

In [ ]:
llm_a = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0.5, max_tokens=500)
agent = ConversationalChatAgent.from_llm_and_tools(llm=llm_a, tools=tools, system_message=CUSTOM_CHATBOT_PREFIX, human_message=CUSTOM_CHATBOT_SUFFIX)
memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True, k=10, chat_memory=cosmos)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, memory=memory, handle_parsing_errors=True)

In [ ]:
# Let's see the custom prompt prefix we created for our brain agent
printmd(agent_chain.agent.llm_chain.prompt.messages[0].prompt.template)

In [ ]:
# Also let's see the Prompt that the Agent uses to talk to the LLM
printmd(agent_chain.agent.llm_chain.prompt.messages[2].prompt.template)

### Let's talk to our GPT Smart Search Engine chat bot now

In [ ]:
# This question should not use any tool, the brain agent should answer it without the use of any tool
printmd(run_agent("hi, how are you doing today?", agent_chain))

In [ ]:
# This question should not use any tool either
printmd(run_agent("what is your name?", agent_chain))

In [ ]:
printmd(run_agent("@bing, I need to take my girlfriend to dinner tonight in downtown Chicago. Please give me options for Italian and Sushi as well", agent_chain))

In [ ]:
printmd(run_agent("@chatgpt, tell me the formula in physics for momentum", agent_chain))

In [ ]:
printmd(run_agent("@docsearch, what can markov chains do?", agent_chain))

In [ ]:
printmd(run_agent("@sqlsearch, How many people died of covid in Texas in 2020?", agent_chain))

In [ ]:
printmd(run_agent("@booksearch, I don't know how to say No to my kids, help me! What kind of boundaries should I set?", agent_chain))

In [ ]:
printmd(run_agent("@bing, How do I cook a chocolate cake?", agent_chain))

In [ ]:
# This question although does not contain instructions for a tool, the brain agent decides what tool to use
printmd(run_agent("What's a good place to dine today in downtown Seoul?", agent_chain))

In [ ]:
# This question many times causes a parsing error, but we can still give the answer using the run_agent function
# which handles the parsing error exception
printmd(run_agent("@chatgpt, can you give me a javascript example of how to trim the spaces of a sentence?", agent_chain))

In [ ]:
# This question should trigger our prompt safety instructions
printmd(run_agent("Tell me a funny joke about the president", agent_chain))

In [ ]:
printmd(run_agent("Thank you for the information, have a good day Jarvis!", agent_chain))

In [ ]:
agent_chain.memory.buffer

# Summary

Great!, We just built the GPT Smart Search Engine!
In this Notebook we created the brain, the decision making Agent that decides what Tool to use to answer the question from the user. This is what was necessary in order to have an smart chat bot.

We can have many tools to accomplish different tasks, including connecting to APIs, dealing with File Systems, and even using Humans as Tools. For more reference see [HERE](https://python.langchain.com/en/latest/modules/agents/tools.html)

# NEXT
It is time now to use all the functions and prompts build so far and build a Web application.
The Next notebook will guide you on how to build:

1) A Bot API Backend
2) A Frontend UI with a Search and Webchat interfaces